## Competition organised by [AnalyticsVidhya](http://analyticsvidhya.com/)

### Problem Statement
The problem is based on the sentiment analysis, we have been given the reviews based on products and we have to predict the its sentiment. Sentiment is classified in two parts 0 and 1 (0 negative and 1 positive).

### Datasets
The dataset contains two features 
1. Label (0 and 1)
2. Tweets

We have provided three dataset train, test and sample submission.

### Metrics used
Here, we have to use f1-score to compete.

#### Import dependencies

In [1]:
#import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report

%matplotlib inline

plt.rcParams['figure.figsize']= (15, 12)
plt.style.use('ggplot')
sns.set(color_codes= True)

#### Read the data

In [2]:
df_train= pd.read_csv('train.csv')
df_test= pd.read_csv('test.csv')
df_samp= pd.read_csv('sample.csv')

#### Data preprocessing

In [3]:
import re

def preprocessing_text_length(tweets):
    num_words= []
    num_char= []
    avg_word_len= []
    num_stopwords= []
    num_special_char= []
    num_upper_cases= []
    num_numerics= []

    for tweet in tweets:
        #remove links 
        #tweet= re.sub(r'(http|https|ftp)://[a-zA-Z0-9\./]+', '', tweet, flags= re.I)
        #tweet = re.sub(r'^https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
        
        tweet= tweet.strip().split()
        
        #num of words
        words= [w for w in tweet]
        num_words.append(len(words))
        
        #num_char
        chars= len(tweet)
        num_char.append(chars)
        
        #num_avg word length
        words_avg_len= [sum(len(w) for w in tweet)/len(words)]
        avg_word_len.append(words_avg_len[0])
        
        #number of stop words
        from nltk.corpus import stopwords
        stop= stopwords.words('english')
        stopword= [w for w in tweet if w in stop]
        num_stopwords.append(len(stopword))
        
        #number of special character
        hastags= [w for w in tweet if w.startswith('#')]
        num_special_char.append(len(hastags))
        
        #number of numerics
        numerics= [w for w in tweet if w.isdigit()]
        num_numerics.append(len(numerics))
        
        #number of upper cases
        upper_cases= [w for w in tweet if w.isupper()]
        num_upper_cases.append(len(upper_cases))
        
    return num_char, num_numerics, num_special_char, num_stopwords, num_upper_cases, num_words, avg_word_len

#### Concatenation of data

In [4]:
df_all= pd.concat([df_train, df_test])

num_chars, num_numeric, num_special_chars, num_stopword, num_upper_case, num_word, avg_words_len= preprocessing_text_length(df_all['tweet'])

df_all['num_chars']= num_chars
df_all['num_numeric']= num_numeric
df_all['num_special_chars']= num_special_chars
df_all['num_stopword']= num_stopword
df_all['num_upper_case']= num_upper_case
df_all['num_word']= num_word
df_all['avg_words_len']= avg_words_len

#### Cleaning of the data

In [5]:
def basic_preprocessing(tweets):
    
    #remove punctuations
    tweets= tweets.str.replace('[^\w\s]', '')
    
    #remove stopwords
    from nltk.corpus import stopwords
    stop= stopwords.words('english')
    
    tweets= tweets.apply(lambda x: ' '.join(w for w in x.split() if w not in stop))
    
    #common word removal
    freq= pd.Series(' '.join(tweets).split()).value_counts()[:10]
    
    freq= list(freq)
    tweets= tweets.apply(lambda x: ' '.join(w for w in x.split() if w not in freq))
    
    #rare word removal
    freq= pd.Series(' '.join(tweets).split()).value_counts()[-10:]
    
    freq= list(freq)
    tweets= tweets.apply(lambda x: ' '.join(w for w in x.split() if w not in freq))
    
    #spelling correction
    #from textblob import TextBlob
    #tweets= tweets.apply(lambda x: str(TextBlob(x).correct()))
    
    #tokenize
    from nltk.tokenize import TreebankWordTokenizer
    tokenizer= TreebankWordTokenizer()
    #tweets= tweets.apply(lambda x: (' '.join(w) for w tokenizer.tokenize(x)))
    
    #stemming
    #from nltk.stem import PorterStemmer
    #stemmer= PorterStemmer()
    #tweets= tweets.apply(lambda x: ' '.join[stemmer.stem(w for w in tokenizer.tokenize(x))])
    
    #lemmatization
    from nltk.stem import WordNetLemmatizer
    lemmatizer= WordNetLemmatizer()
    #tweets= tweets.apply(lambda x: lemmatizer.lemmatize(w) for w in x)
    tweets= tweets.apply(lambda x: tokenizer.tokenize(x))
    tweets= tweets.apply(lambda x: [lemmatizer.lemmatize(w) for w in x])
    tweets= tweets.apply(lambda x: ' '.join(w for w in x))
    
    #convert it into lower case
    tweets= tweets.apply(lambda tweet: tweet.strip().lower())
                                             
    return tweets

tweets= df_all['tweet']
tweets= basic_preprocessing(tweets)

df_all['tweets']= tweets

In [6]:
df_all.head()

,id,label,tweet,num_chars,num_numeric,num_special_chars,num_stopword,num_upper_case,num_word,avg_words_len,tweets
0,1,0.0,#fingerprint #Pregnancy Test https://goo.gl/h1...,13,0,11,0,0,13,8.923077,fingerprint pregnancy test httpsgooglh1mfqv an...
1,2,0.0,Finally a transparant silicon case ^^ Thanks t...,17,0,5,3,1,17,6.764706,finally transparant silicon case thanks uncle ...
2,3,0.0,We love this! Would you go? #talk #makememorie...,15,0,8,1,0,15,7.266667,we love would go talk makememories unplug rela...
3,4,0.0,I'm wired I know I'm George I was made that wa...,17,0,4,2,2,17,5.647059,im wired i know im george i made way iphone cu...
4,5,1.0,What amazing service! Apple won't even talk to...,23,0,0,9,2,23,4.434783,what amazing service apple wont even talk ques...


#### Split the data 

In [9]:
train= df_all[:len(df_train)]
test= df_all[len(df_train):]

### Machine learning modeling

In [10]:
from scipy.sparse import hstack  ##it is used to add the columns in sparse matrix

In [11]:
col_to_add= train.columns.tolist()

In [12]:
col_to_add.remove('id')
col_to_add.remove('label')
col_to_add.remove('tweet')
col_to_add.remove('tweets')

### Applied machine learning algorithms

In [13]:
def ml_modeling(model, train, target, test, countVectorizer= True, tfidfVectorizer= False, col_to_add= col_to_add):
    
    #splitted the dataset in training and validation set (splitted at 25%)
    X_train, X_test, y_train, y_test= train_test_split(train, target, random_state= 5) 
    
    #feature extraction techniques (count vectorizer and tfidf vectorizer)
    if countVectorizer:  
        vect= CountVectorizer().fit(X_train['tweets'])
        X_train_vect= vect.transform(X_train['tweets'])
        X_test_vect= vect.transform(X_test['tweets'])
        test_vect= vect.transform(test['tweets'])
        
        #now add the columns to the sparse matrix using the scipy library
        for col in col_to_add:
            X_train_vect= hstack((X_train_vect, np.array(X_train[col])[:, None]))
            X_test_vect= hstack((X_test_vect, np.array(X_test[col])[:, None]))
            test_vect= hstack((test_vect, np.array(test[col])[:, None]))
        
        print('X_train_vect', X_train_vect.shape)
        print('X_test_vect', X_test_vect.shape)
        print('test_vect', test_vect.shape)
        
        #modeling and prediction
        model.fit(X_train_vect, y_train)
        prediction= model.predict(X_test_vect)
        
        #print accuracies
        train_acc= model.score(X_train_vect, y_train)
        test_acc= accuracy_score(y_test, prediction)
        f1= f1_score(y_test, prediction)
        
        print('Training accuracy: {}' .format(train_acc))
        print('Testing accuracy: {}' .format(test_acc))
        print('f1 score: {}' .format(f1))
        
        print('Classification Report: ')
        print(classification_report(y_test, prediction))
        
    if tfidfVectorizer:
        vect= TfidfVectorizer(min_df= 5, ngram_range= (1, 3)).fit(X_train['tweets'])
        X_train_vect= vect.transform(X_train['tweets'])
        X_test_vect= vect.transform(X_test['tweets'])
        test_vect= vect.transform(test['tweets'])
        
        #now add the columns to the sparse matrix using the scipy library
        for col in col_to_add:
            X_train_vect= hstack((X_train_vect, np.array(X_train[col])[:, None]))
            X_test_vect= hstack((X_test_vect, np.array(X_test[col])[:, None]))
            test_vect= hstack((test_vect, np.array(test[col])[:, None]))
        
        print('X_train_vect', X_train_vect.shape)
        print('X_test_vect', X_test_vect.shape)
        print('test_vect', test_vect.shape)
        
        #modeling and prediction
        model.fit(X_train_vect, y_train)
        prediction= model.predict(X_test_vect)
        
        #print accuracies
        train_acc= model.score(X_train_vect, y_train)
        test_acc= accuracy_score(y_test, prediction)
        f1= f1_score(y_test, prediction)
        
        print('Training accuracy: {}' .format(train_acc))
        print('Testing accuracy: {}' .format(test_acc))
        print('f1 score: {}' .format(f1))
        
        print('Classification Report: ')
        print(classification_report(y_test, prediction))
        
    return model, model.predict(test_vect)

In [14]:
from sklearn.linear_model import LogisticRegression

In [16]:
clf_log= LogisticRegression(class_weight= 'balanced', random_state= 0)

In [17]:
#using count vectorizer
model_1, pred_1= ml_modeling(clf_log, train.drop(['id', 'tweet', 'label'], axis= 1), train['label'],
                             test.drop(['id', 'tweet', 'label'], axis= 1)) ## called the ml function

X_train_vect (5940, 18026)
X_test_vect (1980, 18026)
test_vect (1953, 18026)
Training accuracy: 0.9683501683501684
Testing accuracy: 0.8984848484848484
f1 score: 0.8157653528872594
Classification Report: 
             precision    recall  f1-score   support

        0.0       0.96      0.90      0.93      1475
        1.0       0.76      0.88      0.82       505

avg / total       0.91      0.90      0.90      1980



In [18]:
#using tfidf vectorizer
model_2, pred_2= ml_modeling(clf_log, train.drop(['id', 'tweet', 'label'], axis= 1), train['label'],
                             test.drop(['id', 'tweet', 'label'], axis= 1), False, True)

C:\Users\sngupta\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


X_train_vect (5940, 3189)
X_test_vect (1980, 3189)
test_vect (1953, 3189)
Training accuracy: 0.9104377104377105
Testing accuracy: 0.8909090909090909
f1 score: 0.8128249566724437
Classification Report: 
             precision    recall  f1-score   support

        0.0       0.97      0.88      0.92      1475
        1.0       0.72      0.93      0.81       505

avg / total       0.91      0.89      0.89      1980



##### This function directly return the tweets after vecorize it, there in no need to write it again and again

In [19]:
def vectorizer(train, target, test, countVectorizer= True, tfidfVectorizer= False, col_to_add= col_to_add):
    
    X_train, X_test, y_train, y_test= train_test_split(train, target, random_state= 5)
    
    if countVectorizer:
        vect= CountVectorizer().fit(X_train['tweets'])
        X_train_vect= vect.transform(X_train['tweets'])
        X_test_vect= vect.transform(X_test['tweets'])
        test_vect= vect.transform(test['tweets'])
        
        #now add the columns to the sparse matrix using the scipy library
        for col in col_to_add:
            X_train_vect= hstack((X_train_vect, np.array(X_train[col])[:, None]))
            X_test_vect= hstack((X_test_vect, np.array(X_test[col])[:, None]))
            test_vect= hstack((test_vect, np.array(test[col])[:, None]))
        
        print('X_train_vect', X_train_vect.shape)
        print('X_test_vect', X_test_vect.shape)
        print('test_vect', test_vect.shape)
        
    if tfidfVectorizer:
        vect= TfidfVectorizer(min_df= 5, ngram_range= (1, 3)).fit(X_train['tweets'])
        X_train_vect= vect.transform(X_train['tweets'])
        X_test_vect= vect.transform(X_test['tweets'])
        test_vect= vect.transform(test['tweets'])
        
        #now add the columns to the sparse matrix using the scipy library
        for col in col_to_add:
            X_train_vect= hstack((X_train_vect, np.array(X_train[col])[:, None]))
            X_test_vect= hstack((X_test_vect, np.array(X_test[col])[:, None]))
            test_vect= hstack((test_vect, np.array(test[col])[:, None]))
        
        print('X_train_vect', X_train_vect.shape)
        print('X_test_vect', X_test_vect.shape)
        print('test_vect', test_vect.shape)
        
    return X_train_vect, X_test_vect, test_vect, y_train, y_test

In [20]:
##using count vectorizer
X_train_count, X_test_count, test_count, y_train_c, y_test_c= vectorizer(train.drop(['id', 'tweet', 'label'], axis= 1), train['label'],
                                                    test.drop(['id', 'tweet', 'label'], axis= 1))

X_train_vect (5940, 18026)
X_test_vect (1980, 18026)
test_vect (1953, 18026)


In [21]:
##using tfidf vectorizer
X_train_tfidf, X_test_tfidf, test_tfidf, y_train_t, y_test_t= vectorizer(train.drop(['id', 'tweet', 'label'], axis= 1), train['label'],
                                                    test.drop(['id', 'tweet', 'label'], axis= 1), False, tfidfVectorizer= True)

C:\Users\sngupta\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


X_train_vect (5940, 3189)
X_test_vect (1980, 3189)
test_vect (1953, 3189)


In [22]:
X_train_count.shape, X_test_count.shape, test_count.shape, y_train_c.shape, y_test_c.shape

((5940, 18026), (1980, 18026), (1953, 18026), (5940,), (1980,))

In [23]:
X_train_tfidf.shape, X_test_tfidf.shape, test_tfidf.shape, y_train_c.shape, y_test_t.shape

((5940, 3189), (1980, 3189), (1953, 3189), (5940,), (1980,))

#### Deep learning implementation

Now, here, I'm going to implement the ANN model for the sake of getting the better result.

#### Dependencies related to deep learning

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, Activation
from keras.losses import binary_crossentropy, categorical_crossentropy, sparse_categorical_crossentropy
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


#### Make data for deep learning

In [23]:
from scipy.sparse.csr import csr_matrix

In [24]:
X_train_count= csr_matrix(X_train_count)
X_test_count= csr_matrix(X_test_count)
test_count= csr_matrix(test_count)
#X_test_count_1= X_test_count

### Model declaration
Here, I used
1. Used 3 hidden layer
2. Used hidden nodes (100, 50, 20)
3. Used ReLu and sigmoid activation function
4. Used stochastic gradient decent optimizer
5. Used binary cross entropy loss function

In [25]:
model= Sequential()
model.add(Dense(units= 100, activation= 'relu', input_dim= X_train_count.shape[1], kernel_initializer= 'uniform'))
model.add(Dense(units= 50, activation= 'relu'))
model.add(Dense(units= 20, activation= 'sigmoid'))
model.add(Dense(units= 1, activation= 'sigmoid'))

sgd= SGD(lr= 0.1)

model.compile(optimizer= sgd, loss= 'binary_crossentropy', metrics= ['accuracy'])
model.fit(x= X_train_count, y= y_train_c, batch_size= 128, epochs= 100, class_weight= 'balanced', shuffle= False,
          validation_data= (X_test_count, y_test_c))

y_pred_c= model.predict_classes(X_test_count, batch_size= 128)

#print accuracies
train_acc= model.evaluate(X_train_count, y_train_c, batch_size= 128)
test_acc= accuracy_score(y_test_c, y_pred_c)
f1= f1_score(y_test_c, y_pred_c)

print('Detail of training losses and accuracies: ', train_acc)
print('Testing accuracy: {}' .format(test_acc))
print('f1 score: {}' .format(f1))

y_pred_real_c= model.predict_classes(test_count)

Train on 5940 samples, validate on 1980 samples
Epoch 1/100
5940/5940 [==============================] - 5s 788us/step - loss: 0.4670 - acc: 0.7675 - val_loss: 0.3579 - val_acc: 0.8359
Epoch 2/100
5940/5940 [==============================] - 4s 618us/step - loss: 0.3573 - acc: 0.8370 - val_loss: 0.3396 - val_acc: 0.8434
Epoch 3/100
5940/5940 [==============================] - 4s 595us/step - loss: 0.3439 - acc: 0.8426 - val_loss: 0.3332 - val_acc: 0.8455
Epoch 4/100
5940/5940 [==============================] - 4s 608us/step - loss: 0.3385 - acc: 0.8492 - val_loss: 0.3298 - val_acc: 0.8470
Epoch 5/100
5940/5940 [==============================] - 4s 664us/step - loss: 0.3348 - acc: 0.8510 - val_loss: 0.3266 - val_acc: 0.8530
Epoch 6/100
5940/5940 [==============================] - 4s 721us/step - loss: 0.3317 - acc: 0.8517 - val_loss: 0.3240 - val_acc: 0.8551
Epoch 7/100
5940/5940 [==============================] - 4s 619us/step - loss: 0.3290 - acc: 0.8535 - val_loss: 0.3219 - val_acc: 

5940/5940 [==============================] - 4s 647us/step - loss: 0.2843 - acc: 0.8749 - val_loss: 0.2704 - val_acc: 0.8823
Epoch 61/100
5940/5940 [==============================] - 4s 629us/step - loss: 0.2727 - acc: 0.8778 - val_loss: 0.2769 - val_acc: 0.8803
Epoch 62/100
5940/5940 [==============================] - 3s 585us/step - loss: 0.2756 - acc: 0.8778 - val_loss: 0.2713 - val_acc: 0.8808
Epoch 63/100
5940/5940 [==============================] - 4s 589us/step - loss: 0.2762 - acc: 0.8773 - val_loss: 0.2684 - val_acc: 0.8838
Epoch 64/100
5940/5940 [==============================] - 4s 660us/step - loss: 0.2795 - acc: 0.8756 - val_loss: 0.2718 - val_acc: 0.8803
Epoch 65/100
5940/5940 [==============================] - 4s 659us/step - loss: 0.2777 - acc: 0.8778 - val_loss: 0.2684 - val_acc: 0.8838
Epoch 66/100
5940/5940 [==============================] - 3s 584us/step - loss: 0.2751 - acc: 0.8788 - val_loss: 0.2697 - val_acc: 0.8828
Epoch 67/100
5940/5940 [=======================

##### With two hidden layers and adam optimizer

In [30]:
model= Sequential()
model.add(Dense(units= 50, activation= 'relu', input_dim= X_train_count.shape[1], kernel_initializer= 'uniform'))
model.add(Dense(units= 20, activation= 'relu'))
#model.add(Dense(units= 20, activation= 'relu'))
model.add(Dense(units= 1, activation= 'sigmoid'))

#sgd= SGD(lr= 0.1)

model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])
model.fit(x= X_train_count, y= y_train_c, batch_size= 128, epochs= 5, class_weight= 'balanced', shuffle= False,
          validation_data= (X_test_count, y_test_c))

y_pred_c= model.predict_classes(X_test_count, batch_size= 128)

#print accuracies
train_acc= model.evaluate(X_train_count, y_train_c, batch_size= 128)
test_acc= accuracy_score(y_test_c, y_pred_c)
f1= f1_score(y_test_c, y_pred_c)

print('Detail of training losses and accuracies: ', train_acc)
print('Testing accuracy: {}' .format(test_acc))
print('f1 score: {}' .format(f1))

y_pred_real_c_2= model.predict_classes(test_count)

Train on 5940 samples, validate on 1980 samples
Epoch 1/5
5940/5940 [==============================] - 4s 700us/step - loss: 0.4991 - acc: 0.7407 - val_loss: 0.3440 - val_acc: 0.8682
Epoch 2/5
5940/5940 [==============================] - 4s 619us/step - loss: 0.2856 - acc: 0.8806 - val_loss: 0.2766 - val_acc: 0.8848
Epoch 3/5
5940/5940 [==============================] - 4s 634us/step - loss: 0.2193 - acc: 0.9101 - val_loss: 0.2576 - val_acc: 0.8975
Epoch 4/5
5940/5940 [==============================] - 3s 589us/step - loss: 0.1650 - acc: 0.9421 - val_loss: 0.2516 - val_acc: 0.9010
Epoch 5/5
5940/5940 [==============================] - 1s 189us/step
Detail of training losses and accuracies:  [0.09593365830102754, 0.9705387205387206]
Testing accuracy: 0.9025252525252525
f1 score: 0.8194574368568756


In [34]:
model= Sequential()
model.add(Dense(units= 100, activation= 'relu', input_dim= X_train_count.shape[1], kernel_initializer= 'uniform'))
model.add(Dense(units= 10, activation= 'relu'))
#model.add(Dense(units= 20, activation= 'relu'))
model.add(Dense(units= 1, activation= 'sigmoid'))

#sgd= SGD(lr= 0.1)

model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])
model.fit(x= X_train_count, y= y_train_c, batch_size= 128, epochs= 5, class_weight= 'balanced', shuffle= False,
          validation_data= (X_test_count, y_test_c))

y_pred_c= model.predict_classes(X_test_count, batch_size= 128)

#print accuracies
train_acc= model.evaluate(X_train_count, y_train_c, batch_size= 128)
test_acc= accuracy_score(y_test_c, y_pred_c)
f1= f1_score(y_test_c, y_pred_c)

print('Detail of training losses and accuracies: ', train_acc)
print('Testing accuracy: {}' .format(test_acc))
print('f1 score: {}' .format(f1))

y_pred_real_c_2= model.predict_classes(test_count)

Train on 5940 samples, validate on 1980 samples
Epoch 1/5
5940/5940 [==============================] - 6s 1ms/step - loss: 0.5049 - acc: 0.7254 - val_loss: 0.3579 - val_acc: 0.8490
Epoch 2/5
5940/5940 [==============================] - 6s 936us/step - loss: 0.2920 - acc: 0.8739 - val_loss: 0.2745 - val_acc: 0.8833
Epoch 3/5
5940/5940 [==============================] - 5s 853us/step - loss: 0.2237 - acc: 0.9098 - val_loss: 0.2544 - val_acc: 0.8970
Epoch 4/5
5940/5940 [==============================] - 5s 853us/step - loss: 0.1704 - acc: 0.9386 - val_loss: 0.2451 - val_acc: 0.9015
Epoch 5/5
5940/5940 [==============================] - 1s 238us/step
Detail of training losses and accuracies:  [0.10107676187398458, 0.97003367003367]
Testing accuracy: 0.9025252525252525
f1 score: 0.8187793427230047


### Using tfidf vectorizer

In [56]:
X_train_tfidf= csr_matrix(X_train_tfidf)
X_test_tfidf= csr_matrix(X_test_tfidf)
test_tfidf= csr_matrix(test_tfidf)

In [57]:
model= Sequential()
model.add(Dense(units= 100, activation= 'relu', input_dim= X_train_tfidf.shape[1], kernel_initializer= 'uniform'))
model.add(Dense(units= 50, activation= 'relu'))
model.add(Dense(units= 20, activation= 'relu'))
model.add(Dense(units= 1, activation= 'sigmoid'))

#sgd= SGD(lr= 0.1)

model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])
model.fit(x= X_train_tfidf, y= y_train_t, batch_size= 128, epochs= 15, class_weight= 'balanced', shuffle= True,
          validation_data= (X_test_tfidf, y_test_t))

y_pred_t= model.predict_classes(X_test_tfidf, batch_size= 128)

#print accuracies
train_acc= model.evaluate(X_train_tfidf, y_train_t, batch_size= 128)
test_acc= accuracy_score(y_test_t, y_pred_t)
f1= f1_score(y_test_t, y_pred_t)

print('Detail of training losses and accuracies: ', train_acc)
print('Testing accuracy: {}' .format(test_acc))
print('f1 score: {}' .format(f1))

y_pred_real_1= model.predict_classes(test_tfidf)

Train on 5940 samples, validate on 1980 samples
Epoch 1/15
5940/5940 [==============================] - 3s 479us/step - loss: 0.4140 - acc: 0.7919 - val_loss: 0.3110 - val_acc: 0.8626
Epoch 2/15
5940/5940 [==============================] - 1s 174us/step - loss: 0.3073 - acc: 0.8658 - val_loss: 0.2946 - val_acc: 0.8677
Epoch 3/15
5940/5940 [==============================] - 1s 187us/step - loss: 0.2689 - acc: 0.8887 - val_loss: 0.2706 - val_acc: 0.8914
Epoch 4/15
5940/5940 [==============================] - 1s 174us/step - loss: 0.2246 - acc: 0.9113 - val_loss: 0.2659 - val_acc: 0.8722
Epoch 5/15
5940/5940 [==============================] - 1s 197us/step - loss: 0.1830 - acc: 0.9269 - val_loss: 0.2855 - val_acc: 0.8869
Epoch 6/15
5940/5940 [==============================] - 1s 174us/step - loss: 0.1793 - acc: 0.9251 - val_loss: 0.2646 - val_acc: 0.8838
Epoch 7/15
5940/5940 [==============================] - 1s 176us/step - loss: 0.1471 - acc: 0.9441 - val_loss: 0.2609 - val_acc: 0.8929


In [45]:
model= Sequential()
model.add(Dense(units= 100, activation= 'relu', input_dim= X_train_count.shape[1], kernel_initializer= 'uniform'))
model.add(Dense(units= 50, activation= 'relu'))
model.add(Dense(units= 20, activation= 'relu'))
model.add(Dense(units= 1, activation= 'sigmoid'))

#sgd= SGD(lr= 0.1)

model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])
model.fit(x= X_train_count, y= y_train_c, batch_size= 128, epochs= 10, shuffle= True)#, validation_data= (X_test_count_1, y_test_c))

y_pred_c= model.predict_classes(X_test_count, batch_size= 128)

#print accuracies
train_acc= model.evaluate(X_train_count, y_train_c, batch_size= 128)
test_acc= accuracy_score(y_test_c, y_pred_c)
f1= f1_score(y_test_c, y_pred_c)

print('Detail of training losses and accuracies: ', train_acc)
print('Testing accuracy: {}' .format(test_acc))
print('f1 score: {}' .format(f1))

y_pred_real= model.predict_classes(test_count)

Epoch 1/10
5940/5940 [==============================] - 6s 1ms/step - loss: 0.3571 - acc: 0.8322
Epoch 2/10
5940/5940 [==============================] - 5s 788us/step - loss: 0.2223 - acc: 0.9111
Epoch 3/10
5940/5940 [==============================] - 5s 762us/step - loss: 0.1428 - acc: 0.9520
Epoch 4/10
5940/5940 [==============================] - 5s 837us/step - loss: 0.0806 - acc: 0.9771
Epoch 5/10
5940/5940 [==============================] - 5s 808us/step - loss: 0.0477 - acc: 0.9870
Epoch 6/10
5940/5940 [==============================] - 5s 784us/step - loss: 0.0263 - acc: 0.9944
Epoch 7/10
5940/5940 [==============================] - 5s 823us/step - loss: 0.0173 - acc: 0.9971
Epoch 8/10
5940/5940 [==============================] - 5s 835us/step - loss: 0.0119 - acc: 0.9978
Epoch 9/10
5940/5940 [==============================] - 5s 788us/step - loss: 0.0076 - acc: 0.9992 2s - los - ETA: 0s - loss: 0.0077 - acc: 0.99
Epoch 10/10
5940/5940 [==============================] - 2s 318us

In [61]:
X_train_count.shape, test_count.shape

((5940, 18026), (1953, 22158))

In [47]:
model= Sequential()
model.add(Dense(units= 200, activation= 'relu', input_dim= X_train_tfidf_.shape[1]))
model.add(Dropout(0.1, seed= 0))
model.add(Dense(units= 20, activation= 'relu'))
model.add(Dropout(0.1, seed= 0))
model.add(Dense(units= 1, activation= 'sigmoid'))

model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

model.fit(X_train_tfidf_, y_train_t_, batch_size= 200, epochs= 5, validation_data= (X_test_tfidf_, y_test_t_))

y_pred= model.predict_classes(X_test_tfidf_, batch_size= 200)

train_acc= model.evaluate(X_train_tfidf_, y_train_t_, batch_size= 200)

test_acc= accuracy_score(y_test_t_, y_pred)

f1= f1_score(y_test_t_, y_pred)

print(train_acc)
print(test_acc)
print(f1)

y_pred_real_tf= model.predict_classes(test_tfidf_, batch_size= 32)

Train on 5940 samples, validate on 1980 samples
Epoch 1/5
5940/5940 [==============================] - 3s 563us/step - loss: 0.3893 - acc: 0.8157 - val_loss: 0.3074 - val_acc: 0.8591
Epoch 2/5
5940/5940 [==============================] - 2s 355us/step - loss: 0.3012 - acc: 0.8663 - val_loss: 0.2896 - val_acc: 0.8768
Epoch 3/5
5940/5940 [==============================] - 2s 368us/step - loss: 0.2742 - acc: 0.8845 - val_loss: 0.2666 - val_acc: 0.8859
Epoch 4/5
5940/5940 [==============================] - 2s 362us/step - loss: 0.2402 - acc: 0.9037 - val_loss: 0.2488 - val_acc: 0.8914
Epoch 5/5
5940/5940 [==============================] - 1s 89us/step
[0.1683602602995606, 0.9368686866679978]
0.8994949494949495
0.8131455399061033


### Used ensmble method

In [27]:
from sklearn.ensemble import BaggingClassifier, VotingClassifier, AdaBoostClassifier

In [79]:
clf_bag_log= BaggingClassifier(clf_log, random_state= 0, max_samples= 0.8)

In [80]:
model_5, pred_5= ml_modeling(clf_bag_log, train, train['label'], test)

X_train_vect (5940, 18820)
X_test_vect (1980, 18820)
test_vect (1953, 18820)
Training accuracy: 0.9488215488215488
Testing accuracy: 0.8964646464646465
f1 score: 0.8138056312443233
Classification Report: 
             precision    recall  f1-score   support

        0.0       0.96      0.90      0.93      1475
        1.0       0.75      0.89      0.81       505

avg / total       0.91      0.90      0.90      1980



In [81]:
clf_ada_log= AdaBoostClassifier(clf_log, learning_rate= 0.1)

In [82]:
model_6, pred_6= ml_modeling(clf_ada_log, train, train['label'], test)

X_train_vect (5940, 18820)
X_test_vect (1980, 18820)
test_vect (1953, 18820)
Training accuracy: 0.8604377104377104
Testing accuracy: 0.8641414141414141
f1 score: 0.7726120033812343
Classification Report: 
             precision    recall  f1-score   support

        0.0       0.96      0.85      0.90      1475
        1.0       0.67      0.90      0.77       505

avg / total       0.89      0.86      0.87      1980



In [83]:
clf_vote= VotingClassifier([('log', clf_log), ('bag_log', clf_bag_log)], voting= 'soft')

In [84]:
model_7, pred_7= ml_modeling(clf_vote, train, train['label'], test)

X_train_vect (5940, 18820)
X_test_vect (1980, 18820)
test_vect (1953, 18820)
Training accuracy: 0.9631313131313132
Testing accuracy: 0.8939393939393939
f1 score: 0.8073394495412844
Classification Report: 
             precision    recall  f1-score   support

        0.0       0.95      0.90      0.93      1475
        1.0       0.75      0.87      0.81       505

avg / total       0.90      0.89      0.90      1980



C:\Users\sngupta\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\sngupta\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\sngupta\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [88]:
def ml_modeling(model, train, target, test, countVectorizer= True, tfidfVectorizer= False, col_to_add= col_to_add):
    
    X_train, X_test, y_train, y_test= train_test_split(train, target, random_state= 5)
    
    if countVectorizer:
        vect= CountVectorizer().fit(X_train['tweets'])
        X_train_vect= vect.transform(X_train['tweets'])
        X_test_vect= vect.transform(X_test['tweets'])
        test_vect= vect.transform(test['tweets'])
        
        #now add the columns to the sparse matrix using the scipy library
        for col in col_to_add:
            X_train_vect= hstack((X_train_vect, np.array(X_train[col])[:, None]))
            X_test_vect= hstack((X_test_vect, np.array(X_test[col])[:, None]))
            test_vect= hstack((test_vect, np.array(test[col])[:, None]))
        
        print('X_train_vect', X_train_vect.shape)
        print('X_test_vect', X_test_vect.shape)
        print('test_vect', test_vect.shape)
        
        #modeling and prediction
        model.fit(X_train_vect, y_train)
        prediction= model.predict(X_test_vect)
        
        #print accuracies
        train_acc= model.score(X_train_vect, y_train)
        test_acc= accuracy_score(y_test, prediction)
        f1= f1_score(y_test, prediction)
        
        print('Training accuracy: {}' .format(train_acc))
        print('Testing accuracy: {}' .format(test_acc))
        print('f1 score: {}' .format(f1))
        
        print('Classification Report: ')
        print(classification_report(y_test, prediction))
        
    if tfidfVectorizer:
        vect= TfidfVectorizer(min_df= 5, ngram_range= (1, 3)).fit(X_train['tweets'])
        X_train_vect= vect.transform(X_train['tweets'])
        X_test_vect= vect.transform(X_test['tweets'])
        test_vect= vect.transform(test['tweets'])
        
        #now add the columns to the sparse matrix using the scipy library
        for col in col_to_add:
            X_train_vect= hstack((X_train_vect, np.array(X_train[col])[:, None]))
            X_test_vect= hstack((X_test_vect, np.array(X_test[col])[:, None]))
            test_vect= hstack((test_vect, np.array(test[col])[:, None]))
        
        print('X_train_vect', X_train_vect.shape)
        print('X_test_vect', X_test_vect.shape)
        print('test_vect', test_vect.shape)
        
        #modeling and prediction
        model.fit(X_train_vect, y_train)
        prediction= model.predict(X_test_vect)
        
        #print accuracies
        train_acc= model.score(X_train_vect, y_train)
        test_acc= accuracy_score(y_test, prediction)
        f1= f1_score(y_test, prediction)
        
        print('Training accuracy: {}' .format(train_acc))
        print('Testing accuracy: {}' .format(test_acc))
        print('f1 score: {}' .format(f1))
        
        print('Classification Report: ')
        print(classification_report(y_test, prediction))
        
    return model, model.predict(test_vect)

In [89]:
model_10, pred_10= ml_modeling(clf_bag_log, train, train['label'], test)

X_train_vect (5940, 18026)
X_test_vect (1980, 18026)
test_vect (1953, 18026)
Training accuracy: 0.9484848484848485
Testing accuracy: 0.8984848484848484
f1 score: 0.8161024702653248
Classification Report: 
             precision    recall  f1-score   support

        0.0       0.96      0.90      0.93      1475
        1.0       0.76      0.88      0.82       505

avg / total       0.91      0.90      0.90      1980



### Finally, I got the best result using deep learning with f1-score 0.81945